In [1]:
from bs4 import BeautifulSoup
import json
import os

In [2]:
def get_categories(soup):
    """
    ['今日のご飯・おかず/野菜のおかず/旬野菜（秋・冬）/さつまいも',
    '今日のご飯・おかず/野菜のおかず/旬野菜（秋・冬）/れんこん']
    """
    tag_div = soup.find(id='category')
    if tag_div is None:
        return None
    tags_li = tag_div.find_all(class_='with_icon_arrow')
    categories = []
    for tag_li in tags_li:
        tags_a = tag_li.find_all('a')
        hierarchical_category = '/'.join(tag_a.get_text() for tag_a in tags_a)
        categories.append(hierarchical_category)
    return categories

def get_rankings(soup):
    """
    ['さつまいも 2位', 'れんこん 4位', 'デパ地下 1位']
    """
    tag_div = soup.find(id='keyword_ranking_wrapper')
    if tag_div is None:
        return None
    tags_li = tag_div.find_all(class_='with_icon_arrow')
    rankings = []
    for tag_li in tags_li:
        category = tag_li.find(class_='keyword_ranking_link').get_text()
        ranking = tag_li.find(class_='bold rank').get_text().strip('\n')
        rankings.append('{} {}'.format(category, ranking))
    return rankings

def get_related_keywords(soup):
    """
    ['さつまいも', 'れんこん', 'れんこん さつまいも', 'れんこん さつまいも デパ地下']
    """
    tag_ul = soup.find(class_='related_keywords')
    if tag_ul is None:
        return None
    tags_li = tag_ul.find_all(class_='with_icon_arrow')
    related_keywords = [tag_li.find('a').get_text() for tag_li in tags_li]
    return related_keywords

In [3]:
def scrape_recipe(path_to_html):
    """
    cookpadのレシピのhtmlから
    各属性を取得し、辞書として返す
    """
    # html中の先頭のメタデータ？のような部分に
    # json形式でほとんどのデータが掲載されている
    with open(path_to_html) as f:
        soup = BeautifulSoup(f)
    text = soup.find(type="application/ld+json").get_text().strip('\n')
    d = json.loads(text)
    
    # レシピID、「コツ・ポイント」、「レシピの生い立ち」、
    # カテゴリー、ランキング、関連キーワードを抽出
    id_ = soup.find('span', class_='recipe_id').get_text().strip('\n').lstrip('レシピID: ')
    advice = soup.find(id='advice').get_text().strip('\n')
    history = soup.find(id='history').get_text().strip('\n')
    categories = get_categories(soup)
    rankings = get_rankings(soup)
    related_keywords = get_related_keywords(soup)
    
    d['recipe_id'] = id_
    d['advice'] = advice
    d['history'] = history
    d['categories'] = categories
    d['rankings'] = rankings
    d['related_keywords'] = related_keywords
    
    return d

In [4]:
scrape_recipe('recipes/page1.html')

{'@context': 'http://schema.org',
 '@type': 'Recipe',
 'name': '簡単！激ウマ！甘辛の豚バラ大根',
 'author': {'@type': 'Person', 'name': 'ナウちゃん'},
 'image': 'https://img.cpcdn.com/recipes/2543737/464x260/49ba01cc5d66b8e44e415fd473e3d5ce.jpg?u=8848832&p=1397198218',
 'datePublished': '2014-03-14',
 'description': '殿堂入り！つくれぽ3300件感謝です！砂糖を加えてコクを出し、ごま油風味に仕上げました。',
 'recipeYield': '4人分',
 'recipeIngredient': ['大根 1/3本',
  '豚バラ肉 150g',
  'ごま油(炒め用) 大さじ1',
  '●醤油・みりん・砂糖 各大さじ2',
  '●ほんだし(顆粒) 小さじ1/2',
  '●水 200cc',
  '万能ねぎ(小口切り) 適量'],
 'recipeInstructions': ['下ごしらえ\n\n・肉:3cmに切る\n・大根:厚さ1cmの銀杏切り\n・万能ねぎ:小口切り\n・●印の調味料を合わせる\n',
  '肉を炒める\n\n鍋にごま油しき、豚バラ肉を入れ、中火で炒める。',
  '大根を炒める\n\n豚バラ肉に火が通ったら、大根を入れ、中火で炒める。',
  '味付け\n\n●印の合わせ調味料を入れて、味付けをする。',
  '煮込み\n\n強火で灰汁を採りながら煮詰めて、煮汁が少し残る程度まで煮込む。',
  '盛り付け\n\n器に盛り付けて、万能ねぎを振り掛けて出来上がり。'],
 'cookTime': 'PT30M',
 'recipe_id': '2543737',
 'advice': '・特に難しい点はありません。しいて言えば、煮詰める際には、煮汁が少なくなるまで煮詰めるので、焦がさないように注意してくさい＾＾甘さ控え目希望の方は砂糖大さじ１もオススメです。',
 'history': '砂糖を加えてコクを出すことで、我が家の味付け風にしてみました。今では家族みん

In [5]:
n_search_results = 2
NRECIPE_PER_SEARCH_RESULT = 10
n_recipes = NRECIPE_PER_SEARCH_RESULT * n_search_results
paths = ['recipes/page{}.html'.format(n) for n in range(1, n_recipes+1)]

In [6]:
os.makedirs('recipe_jsons', exist_ok=True)

for path in paths:
    d = scrape_recipe(path)
    recipe_id = d['recipe_id']
    # 結果をjson形式で保存
    with open('recipe_jsons/recipe{}.json'.format(recipe_id), 'w') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)  